In [14]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, Audio
import librosa
import random
import whisper
import adapters

- randomly select a single-channel audio stream from the meetings
- compile a list of these meetings
- create a df with their transcription files
- Divide them into train and eval sets

In [10]:
train_dir =  '/ceph/dpandya/notsofar/train_set/240825.1_train/MTG/'
eval_dir = '/ceph/dpandya/notsofar/eval_set/240825.1_eval_full_with_GT/MTG/'
train_meetings = os.listdir('/ceph/dpandya/notsofar/train_set/240825.1_train/MTG/')
eval_meetings = os.listdir('/ceph/dpandya/notsofar/eval_set/240825.1_eval_full_with_GT/MTG/')

In [20]:
train_files = []
transcription_files = []
mic = []
# Iterate through all the train meeting dir and choosing a random single channel audio stream
for meet in train_meetings:
    stream_list = []
    # Collecting all the streams for a meeting
    for stream in os.listdir(os.path.join(train_dir, meet)):
        if os.path.isdir(os.path.join(train_dir, meet, stream)):
            if stream.split('_')[0] == 'sc':
                stream_list.append(os.path.join(train_dir, meet, stream, 'ch0.wav'))
    random_ = random.choice(stream_list)
    
    mic.append(random_.split('/')[-2])
    train_files.append(random_)
    transcription_files.append(os.path.join(train_dir, meet, 'gt_transcription.json'))
    

In [31]:
# Do the same process for the eval files, choosing a random single channel audio stream
eval_files = []
eval_trans = []
eval_mic = []
# Iterate through all the eval meeting dir and choosing a random single channel audio stream
for meet in eval_meetings:
    stream_list = []
    # Collecting all the streams for a meeting
    if os.path.isdir(os.path.join(eval_dir, meet)):
        for stream in os.listdir(os.path.join(eval_dir, meet)):
            if os.path.isdir(os.path.join(eval_dir, meet, stream)):
                if stream.split('_')[0] == 'sc':
                    stream_list.append(os.path.join(eval_dir, meet, stream, 'ch0.wav'))
        random_ = random.choice(stream_list)
    
        eval_mic.append(random_.split('/')[-2])
        eval_files.append(random_)
        eval_trans.append(os.path.join(eval_dir, meet, 'gt_transcription.json'))

In [35]:
train_df = pd.DataFrame()
train_df['audio_files'] = train_files
train_df['transcription_file'] = transcription_files
train_df['mic'] = mic

test_df = pd.DataFrame()
test_df['audio_files'] = eval_files
test_df['transcription_files'] = eval_trans
test_df['mic'] = eval_mic

In [46]:
train_df.to_csv('/ceph/dpandya/notsofar/newNotsofar/train.csv', index=False)
test_df.to_csv('/ceph/dpandya/notsofar/newNotsofar/test.csv', index=False)